In [1]:
import cv2
import numpy as np
import scipy.ndimage
import scipy.stats
import time
import multiprocessing
import ImportNotebook
import GenerateDaugmanTemplateSet
import DaugmanIrisCore
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import copy as cp


mypool =  multiprocessing.Pool()
#？这几个变量的含义
imageWorkingWidth = 150.0
irisRangeLB = 0.16
irisRangeUB = 0.24
irisTemplateSet = []
irisWeights = []
irisRange = []
irisRadiiLog = []
efficient=np.array([1,0.2,0.2,0.9])

def regulateIrisRange():
    from scipy.stats import norm
    #得到随机序列的期望和标准差，更改原理是啥？
    (mu, sigma) = norm.fit(irisRadiiLog)
    global irisRangeLB
    global irisRangeUB
    irisRangeLB = mu - 2*sigma
    irisRangeUB = mu + 2*sigma
    #irisRangeLB = mu - sigma
    #irisRangeUB = mu + 3*sigma
    generateTemplateSet()
    #generateTemplateSet2()#新模板不用再次生成，用旧的就好

def generateTemplateSet():
    global irisTemplateSet
    global irisWeights
    global irisRange
    irisRange = np.arange(round(irisRangeLB*imageWorkingWidth),round(irisRangeUB*imageWorkingWidth)+1,1)
    irisRingWidth = 1
    GenerateDaugmanTemplateSet.GenerateDaugmanTemplateSet.pool = mypool
    irisTemplateSet,irisWeights = GenerateDaugmanTemplateSet.GenerateDaugmanTemplateSet(irisRingWidth,irisRange)
    #print 'irisTemplateSet'
    #print len(irisTemplateSet[0])
    #print len(irisTemplateSet[1])

def generateTemplateSet2():
    global irisTemplateSet2
    global irisWeights2
    global irisRange2
    #irisRange2 = np.arange(round(irisRangeLB*imageWorkingWidth),round(irisRangeUB*imageWorkingWidth)+1,1)
    irisRange2 = np.array([24.,25.,26.,27.,28.,29.,30.,31.,32.,33.,34.,35.])
    irisRingWidth2 = 1
    irisTemplateSet2,irisWeights2,index2 = GenerateDaugmanTemplateSet.GenerateDaugmanTemplateSet2(irisRingWidth2,irisRange2)
    #print 'irisTemplateSet2'
    #print len(irisTemplateSet2)
    
def EqualizeRGBImage(img):
    #转化成YUV通道
    ycrcb = cv2.cvtColor(img,cv2.COLOR_RGB2YCR_CB)
    #灰度图直方图均衡，第0层是亮度
    ycrcb[:,:,0] = cv2.equalizeHist(ycrcb[:,:,0])
    #再转化回来
    return cv2.cvtColor(ycrcb,cv2.COLOR_YCR_CB2RGB)


def control(leftEyeRectified,rightEyeRectified,frameCount):
    #需要根据本帧的图片，圆心，半径来得到上下比例
    #图片是leftEyeRectified,rightEyeRectified,圆心是leyecenterInRect，reyecenterInRect，半径lIrisRadius，rIrisRadius
    global information
    global leftEyeInfo
    global rightEyeInfo
    eyecenterInRect=[]
    IrisRadius=[]
    diff=[]
    DaugmanCenter=[]
    DaugmanRadius=[]
    EyeFiltered=[]
    if frameCount < 30:#使用原方法
        leyecenterInRect,lIrisRadius,diffLeft,leftDaugmanCenter,leftDaugmanRadius,leftEyeFiltered = DaugmanIrisLocalization(leftEyeRectified,frameCount,1)
        reyecenterInRect,rIrisRadius,diffRight,rightDaugmanCenter,rightDaugmanRadius,rightEyeFiltered = DaugmanIrisLocalization(rightEyeRectified,frameCount,2)
    else:
        if frameCount%10==0:#每10帧使用原模板
            succ,leyecenterInRect,lIrisRadius,diffLeft,leftDaugmanCenter,leftDaugmanRadius,leftEyeFiltered,information = DaugmanIrisLocalization1(leftEyeRectified,frameCount,1)
            if succ==0:
                return 0,eyecenterInRect,IrisRadius,diff,DaugmanCenter,DaugmanRadius,EyeFiltered
            leftEyeInfo=cp.deepcopy(information)
            succ,reyecenterInRect,rIrisRadius,diffRight,rightDaugmanCenter,rightDaugmanRadius,rightEyeFiltered,information = DaugmanIrisLocalization1(rightEyeRectified,frameCount,2)
            if succ==0:
                return 0,eyecenterInRect,IrisRadius,diff,DaugmanCenter,DaugmanRadius,EyeFiltered
            rightEyeInfo=cp.deepcopy(information)
            #左右眼的模板差异太大的时候要不要再来一帧？？？？？？？？？？？？？？？？？
            print 'info_of_eye'
            print leftEyeInfo
            print rightEyeInfo
        else:#其余使用新的模板
            succ,leyecenterInRect,lIrisRadius,diffLeft,leftDaugmanCenter,leftDaugmanRadius,leftEyeFiltered,information = DaugmanIrisLocalization2(leftEyeRectified,leftEyeInfo,frameCount,1)
            if succ==0:
                return 0,eyecenterInRect,IrisRadius,diff,DaugmanCenter,DaugmanRadius,EyeFiltered
            leftEyeInfo=cp.deepcopy(information)
            succ,reyecenterInRect,rIrisRadius,diffRight,rightDaugmanCenter,rightDaugmanRadius,rightEyeFiltered,information = DaugmanIrisLocalization2(rightEyeRectified,rightEyeInfo,frameCount,2)
            if succ==0:
                return 0,eyecenterInRect,IrisRadius,diff,DaugmanCenter,DaugmanRadius,EyeFiltered
            rightEyeInfo=cp.deepcopy(information)
            print 'info_of_eye'
            print leftEyeInfo
            print rightEyeInfo
    #leyecenterInRect,lIrisRadius,diffLeft,leftDaugmanCenter,leftDaugmanRadius,leftEyeFiltered = DaugmanIrisLocalization(leftEyeRectified,frameCount,1)
    #reyecenterInRect,rIrisRadius,diffRight,rightDaugmanCenter,rightDaugmanRadius,rightEyeFiltered = DaugmanIrisLocalization(rightEyeRectified,frameCount,2)
    
    
    eyecenterInRect.append(leyecenterInRect)
    eyecenterInRect.append(reyecenterInRect)
    IrisRadius.append(lIrisRadius)
    IrisRadius.append(rIrisRadius)
    diff.append(diffLeft)
    diff.append(diffRight)
    DaugmanCenter.append(leftDaugmanCenter)
    DaugmanCenter.append(rightDaugmanCenter)
    DaugmanRadius.append(leftDaugmanRadius)
    DaugmanRadius.append(rightDaugmanRadius)
    EyeFiltered.append(leftEyeFiltered)
    EyeFiltered.append(rightEyeFiltered)
    return 1,eyecenterInRect,IrisRadius,diff,DaugmanCenter,DaugmanRadius,EyeFiltered

def DaugmanIrisLocalization(img,count,num): 
    countStr='%d'%count
    numStr='%d'%num
    #cv2.imwrite('/Users/ly/Code/picBefore/eye'+countStr+'_'+numStr+'.jpg',img)
    #做了亮度的处理，为了更好的找到边界
    imgeq = EqualizeRGBImage(img)
    #cv2.imshow('test1',img)
    imglarge = cv2.resize(imgeq,(np.int(imageWorkingWidth),np.int(np.ceil(imageWorkingWidth/imgeq.shape[1]*imgeq.shape[0]))),interpolation=cv2.INTER_LINEAR)
    #cv2.imshow('test2',imglarge)
    #保护边缘的平滑滤波器bilateralFilter
    imgfiltered = cv2.bilateralFilter(imglarge,5,150,50)
    #cv2.imshow('test3',imgfiltered)
    #print imgfiltered.shape
    #cv2.imwrite('/Users/ly/Code/picAfter/eye'+countStr+'_'+numStr+'.jpg',imgfiltered)
    imggrey = cv2.cvtColor(imgfiltered,cv2.COLOR_RGB2GRAY)
    #cv2.imshow('test4',imggrey)
    
    print 'irisRange:'
    print irisRange
    
    covstackDiffSet = []
    covstackdiffmaxmaps = np.zeros(list(imggrey.shape) + [len(irisTemplateSet)])
    covstackdiffmaxradii = np.zeros(list(imggrey.shape) + [len(irisTemplateSet)])
    #print '-' *40
    #print imggrey.shape      #(104,150)或者(105,150)或者(106,150)
    #print list(imggrey.shape) #[104,150]
    #print [len(irisTemplateSet)]    #[2]
    #print list(imggrey.shape) + [len(irisTemplateSet)]  #[104,150,2]
    #print covstackdiffmaxmaps.shape  #(104,150,2)
    #print covstackdiffmaxmaps
    #print '-' *40
        
    results = [mypool.apply_async(DaugmanIrisCore.DaugmanIrisCore,(irisTemplates,imggrey)) for irisTemplates in irisTemplateSet]
    parallelResults = [r.get() for r in results]
        
    for s in xrange(len(parallelResults)):
        covstackdiffmaxmaps[:,:,s] = parallelResults[s][0]
        covstackdiffmaxradii[:,:,s] = parallelResults[s][1]
        covstackDiffSet.append(parallelResults[s][2])
    
    covstackdiffmaxsum = 0
    for s in xrange(len(irisTemplateSet)):
        covstackdiffmaxsum += covstackdiffmaxmaps[:,:,s]*irisWeights[s]
    covstackdiffmaxsum **= 3
    covstackdiffmaxsumfiltered = scipy.ndimage.gaussian_filter(covstackdiffmaxsum,6,0,None,mode='nearest')
    DaugmanIrisCenter = np.unravel_index(np.argmax(covstackdiffmaxsumfiltered),covstackdiffmaxsumfiltered.shape)
    DaugmanIrisRadius = irisRange[np.round(scipy.stats.nanmedian(covstackdiffmaxradii[DaugmanIrisCenter[0],DaugmanIrisCenter[1],:]))]
    global irisRadiiLog
    irisRadiiLog.append(DaugmanIrisRadius/imageWorkingWidth)
    DaugmanIrisCenter = DaugmanIrisCenter[::-1]
    #分别计算出眼珠中心的坐标和黑眼仁的半径
    eyecenter = np.float64(DaugmanIrisCenter) * (img.shape[1]/imageWorkingWidth)
    eyeiris =   np.float64(DaugmanIrisRadius) * (img.shape[1]/imageWorkingWidth)
    print type(DaugmanIrisCenter)
    print DaugmanIrisCenter
    print type(DaugmanIrisCenter[0])
    print type(DaugmanIrisRadius)
    print DaugmanIrisRadius
    #cv2.circle(imggrey,tuple(np.int32(np.squeeze(DaugmanIrisCenter))),1,(0,0,255))
    #cv2.circle(imggrey,tuple(np.int32(np.squeeze(DaugmanIrisCenter))),np.int32(DaugmanIrisRadius),(0,0,255))
    #cv2.imshow('hello',imggrey)
    return eyecenter,eyeiris,covstackdiffmaxsum,DaugmanIrisCenter,DaugmanIrisRadius,imgfiltered

def fillHoleInBinary(Img,binaryImg):
    contours,hierarchy=cv2.findContours(binaryImg,cv2.RETR_LIST,cv2.CHAIN_APPROX_NONE)
    if len(contours)!=0:
        for i in xrange(0,len(contours)):
            if len(contours[i])<50:
                cv2.drawContours(Img,contours,i,(0,0,0),-1)
    else:
        print 'no edge is detected!'
    return Img

def UseKmeans(noHoleImg,Img):
    l1=np.reshape(noHoleImg[:,:,0].T,(Img.shape[0]*Img.shape[1],1))
    l2=np.reshape(noHoleImg[:,:,1].T,(Img.shape[0]*Img.shape[1],1))
    l3=np.reshape(noHoleImg[:,:,2].T,(Img.shape[0]*Img.shape[1],1))
    x=np.reshape(np.tile(np.arange(Img.shape[0]),Img.shape[1]),(l1.shape[0],1))
    y=np.reshape(np.arange(Img.shape[1]).repeat(Img.shape[0]),(l1.shape[0],1))
    data=np.hstack((l1*efficient[0],l2*efficient[1],l3*efficient[2],x*efficient[3],y*efficient[3]))
    #the first time to operate kmeans
    kmeans=KMeans(n_clusters=2,random_state=0).fit(data)
    labels=kmeans.predict(data)
    #find the XXX
    if np.mean(data[np.where(labels==1),0:3])>np.mean(data[np.where(labels==0),0:3]):
        ti=0
    else:
        ti=1
    labels[np.where(labels!=ti)]=4
    newdata=data[np.where(labels==ti)]
    newdataIndex=np.where(labels==ti)[0]
    #the second time to operate kmeans
    kmeans=KMeans(n_clusters=2,random_state=0).fit(newdata)
    newlabels=kmeans.predict(newdata)
    if np.mean(newdata[np.where(newlabels==1),0:3])>np.mean(newdata[np.where(newlabels==0),0:3]):
        labels[newdataIndex[np.where(newlabels==1)[0]]]=0
        labels[newdataIndex[np.where(newlabels==0)[0]]]=1
    else:
        labels[newdataIndex[np.where(newlabels==0)[0]]]=0
        labels[newdataIndex[np.where(newlabels==1)[0]]]=1

    labels[np.where(labels==0)]=3
    newdata=data[np.where(labels==1)]
    newdataIndex=np.where(labels==1)[0]
    kmeans=KMeans(n_clusters=2,random_state=0).fit(newdata)
    newlabels=kmeans.predict(newdata)
    if np.mean(newdata[np.where(newlabels==1),0])>np.mean(newdata[np.where(newlabels==0),0]):
        labels[newdataIndex[np.where(newlabels==1)[0]]]=0
        labels[newdataIndex[np.where(newlabels==0)[0]]]=1
    else:
        labels[newdataIndex[np.where(newlabels==0)[0]]]=0
        labels[newdataIndex[np.where(newlabels==1)[0]]]=1
    
    labels[np.where(labels==0)]=3
    newdata=data[np.where(labels==1)]
    newdataIndex=np.where(labels==1)[0]
    kmeans=KMeans(n_clusters=2,random_state=0).fit(newdata)
    newlabels=kmeans.predict(newdata)
    if np.mean(newdata[np.where(newlabels==1),0])>np.mean(newdata[np.where(newlabels==0),0]):
        labels[newdataIndex[np.where(newlabels==1)[0]]]=0
        labels[newdataIndex[np.where(newlabels==0)[0]]]=1
    else:
        labels[newdataIndex[np.where(newlabels==0)[0]]]=0
        labels[newdataIndex[np.where(newlabels==1)[0]]]=1
    labels=np.uint8(255-np.reshape(labels,(Img.shape[1],Img.shape[0])).T*51)
    #cv2.imshow('kmenas',labels)
    #cv2.waitKey()
    labels[np.where(labels!=204)]=0
    labels[np.where(labels==204)]=255
    return labels

#center:<type 'tuple'><type 'numpy.int64'>    radius:<type 'numpy.float64'>
def caculateShelterRate(center,radius,img,count,lor):
    topShelter=[]
    bottomShelter=[]
    radius=int(round(radius))
    topedge=center[1]-radius
    bottomedge=center[1]+radius
    print img.shape
    print 'radius=%d'%(radius)
    print 'topedge=%d'%(topedge)
    print 'bottomedge=%d'%(bottomedge)
    topPos=topedge-1
    bottomPos=bottomedge+1
    
    sumData=np.int64(np.sum(img[:,center[0]-4:center[0]+4],axis=1))
    print 'sumData'
    print sumData
    diffData=np.diff(sumData)
    print diffData
    topPos=np.where(diffData==max(diffData))[0][-1]
    bottomPos=np.where(diffData==min(diffData))[0][0]
    
    temp2Img=cp.deepcopy(img)
    temp2Img[topPos:bottomPos+1,center[0]]=100
    #cv2.imshow('temp2Img_%d_%d'%(count,lor),temp2Img)
    #cv2.waitKey()
    
    out=img[topedge:bottomedge+1,center[0]]
    tempImg=cp.deepcopy(img)
    tempImg[topedge:bottomedge+1,center[0]]=100
    #cv2.imshow('img_%d_%d'%(count,lor),tempImg)
    #cv2.waitKey()
    print out
    print topPos
    print bottomPos
    topShelter.append(topPos-topedge+1)
    #topShelter.append(topPos-topedge+2)
    bottomShelter.append(bottomedge-bottomPos+1)
    #bottomShelter.append(bottomedge-bottomPos+2)
    print 'topShelter'
    print topShelter
    print 'bottomShelter'
    print bottomShelter
    
    return topShelter,bottomShelter
    
def DaugmanIrisLocalization1(img,count,lor):
    print '-'
    print 'DaugmanIrisLocalization1'
    if lor==1:
        print '左眼'
    else:
        print '右眼'
    imgeq = EqualizeRGBImage(img)
    imglarge = cv2.resize(imgeq,(np.int(imageWorkingWidth),np.int(np.ceil(imageWorkingWidth/imgeq.shape[1]*imgeq.shape[0]))),interpolation=cv2.INTER_LINEAR)
    imgfiltered = cv2.bilateralFilter(imglarge,5,150,50)
    imggrey = cv2.cvtColor(imgfiltered,cv2.COLOR_RGB2GRAY)
    
    info=[]
    covstackDiffSet = []
    covstackdiffmaxmaps = np.zeros(list(imggrey.shape) + [len(irisTemplateSet)])
    covstackdiffmaxradii = np.zeros(list(imggrey.shape) + [len(irisTemplateSet)])
        
    results = [mypool.apply_async(DaugmanIrisCore.DaugmanIrisCore,(irisTemplates,imggrey)) for irisTemplates in irisTemplateSet]
    parallelResults = [r.get() for r in results]
        
    for s in xrange(len(parallelResults)):
        covstackdiffmaxmaps[:,:,s] = parallelResults[s][0]
        covstackdiffmaxradii[:,:,s] = parallelResults[s][1]
        covstackDiffSet.append(parallelResults[s][2])
    
    covstackdiffmaxsum = 0
    for s in xrange(len(irisTemplateSet)):
        covstackdiffmaxsum += covstackdiffmaxmaps[:,:,s]*irisWeights[s]
    covstackdiffmaxsum **= 3
    covstackdiffmaxsumfiltered = scipy.ndimage.gaussian_filter(covstackdiffmaxsum,6,0,None,mode='nearest')
    DaugmanIrisCenter = np.unravel_index(np.argmax(covstackdiffmaxsumfiltered),covstackdiffmaxsumfiltered.shape)
    DaugmanIrisRadius = irisRange[np.round(scipy.stats.nanmedian(covstackdiffmaxradii[DaugmanIrisCenter[0],DaugmanIrisCenter[1],:]))]
    global irisRadiiLog
    irisRadiiLog.append(DaugmanIrisRadius/imageWorkingWidth)
    DaugmanIrisCenter = DaugmanIrisCenter[::-1]
    #分别计算出眼珠中心的坐标和黑眼仁的半径
    eyecenter = np.float64(DaugmanIrisCenter) * (img.shape[1]/imageWorkingWidth)
    eyeiris =   np.float64(DaugmanIrisRadius) * (img.shape[1]/imageWorkingWidth)
    
    ret,imgBinary=cv2.threshold(imggrey,70,255,cv2.THRESH_BINARY)
    imgNoHole=fillHoleInBinary(imgfiltered,imgBinary)
    #cv2.imshow('imgNoHole_%d_%d'%(count,lor),imgNoHole)
    imgKmeans=UseKmeans(imgNoHole,imggrey)
    if imgKmeans[np.where(imgKmeans==255)].shape[0]<1200:
        return 0,eyecenter,eyeiris,covstackdiffmaxsum,DaugmanIrisCenter,DaugmanIrisRadius,imgfiltered,info
    #计算出上下遮挡比例
    topShelterRate,bottomShelterRate=caculateShelterRate(DaugmanIrisCenter,DaugmanIrisRadius,imgKmeans,count,lor)
    
    info.append(DaugmanIrisRadius)
    info.append(topShelterRate)
    info.append(bottomShelterRate)
    
    return 1,eyecenter,eyeiris,covstackdiffmaxsum,DaugmanIrisCenter,DaugmanIrisRadius,imgfiltered,info

def DaugmanIrisLocalization2(img,info,count,lor):
    print 'DaugmanIrisLocalization2'
    if lor==1:
        print '左眼'
    else:
        print '右眼'
    imgeq = EqualizeRGBImage(img)
    imglarge = cv2.resize(imgeq,(np.int(imageWorkingWidth),np.int(np.ceil(imageWorkingWidth/imgeq.shape[1]*imgeq.shape[0]))),interpolation=cv2.INTER_LINEAR)
    imgfiltered = cv2.bilateralFilter(imglarge,5,150,50)
    imggrey = cv2.cvtColor(imgfiltered,cv2.COLOR_RGB2GRAY)
    #cv2.imshow('imggrey_%d_%d'%(count,lor),imggrey)
    #cv2.waitKey()
    
    #irisRange2
    #templateIndex1=np.int64(info[0]-irisRange2[0])
    templateIndex2=info[1]
    templateIndex3=info[2]
    #print ')'*40
    #print templateIndex1
    #print templateIndex2
    #print templateIndex3
    #print 'irisTemplateSet2'
    #print len(irisTemplateSet2)
    #print len(irisTemplateSet2[0])
    #print len(irisTemplateSet2[0][0])
    myTemp=[]
    irisWeightsThis=[]
    print 'irisRange:'
    print irisRange
    print len(irisTemplateSet2)
    #if (np.int32(k)-np.int32(irisRange2[0]))>=len(irisTemplateSet2):
    #    eyecenter,eyeiris,covstackdiffmaxsum,DaugmanIrisCenter,DaugmanIrisRadius,imgfiltered,info=DaugmanIrisLocalization1(img,count,lor)
    #    return eyecenter,eyeiris,covstackdiffmaxsum,DaugmanIrisCenter,DaugmanIrisRadius,imgfiltered,info
    for i in xrange(0,len(templateIndex2)):
        #print 'i=%d'%(i)
        for j in xrange(0,len(templateIndex3)):
            #print 'j=%d'%(j)
            mytemplate=[]
            for k in irisRange:
                #print 'k=%d'%(k)
                #对于超过模板上下边界的情况，用边界值：例如上半边最小截取0，最大截取半径
                index1=np.int32(k)-np.int32(irisRange2[0])
                index2=templateIndex2[i]
                index3=templateIndex3[j]
                if index2<0:
                    index2=0
                if index2>=len(irisTemplateSet2[index1]):
                    index2=len(irisTemplateSet2[index1])-1
                if index3<0:
                    index3=0
                if index3>=len(irisTemplateSet2[index1][index2]):
                    index3=len(irisTemplateSet2[index1][index2])-1
                    
                print '1=%d,2=%d,3=%d'%(index1,index2,index3)
                mytemplate.append(irisTemplateSet2[index1][index2][index3])    
                #cv2.imshow('tempalte',irisTemplateSet2[np.int32(k)-np.int32(irisRange2[0])][templateIndex2[i]][templateIndex3[j]]*255)
                #cv2.waitKey()
                print 'append'
            myTemp.append(mytemplate)
            irisWeightsThis.append(1)
    #print 'myTemp'
    #print len(myTemp)
    #print len(myTemp[0])
    
    info=[]
    covstackDiffSet = []
    covstackdiffmaxmaps = np.zeros(list(imggrey.shape) + [len(myTemp)])
    covstackdiffmaxradii = np.zeros(list(imggrey.shape) + [len(myTemp)])
    print '1=1'
        
    results = [mypool.apply_async(DaugmanIrisCore.DaugmanIrisCore,(irisTemplates,imggrey)) for irisTemplates in myTemp]
    parallelResults = [r.get() for r in results]
    
    #print len(parallelResults)
        
    for s in xrange(len(parallelResults)):
        covstackdiffmaxmaps[:,:,s] = parallelResults[s][0]
        covstackdiffmaxradii[:,:,s] = parallelResults[s][1]
        covstackDiffSet.append(parallelResults[s][2])
    print '2=2'
    covstackdiffmaxsum = 0
    
    for s in xrange(len(myTemp)):
        covstackdiffmaxsum += covstackdiffmaxmaps[:,:,s]*irisWeightsThis[s]
    print '3=3'
    covstackdiffmaxsum **= 3
    print '4=4'
    covstackdiffmaxsumfiltered = scipy.ndimage.gaussian_filter(covstackdiffmaxsum,6,0,None,mode='nearest')
    print '5=5'
    DaugmanIrisCenter = np.unravel_index(np.argmax(covstackdiffmaxsumfiltered),covstackdiffmaxsumfiltered.shape)
    print '6=6'
    DaugmanIrisRadius = irisRange[np.round(scipy.stats.nanmedian(covstackdiffmaxradii[DaugmanIrisCenter[0],DaugmanIrisCenter[1],:]))]
    print '7=7'
    global irisRadiiLog
    irisRadiiLog.append(DaugmanIrisRadius/imageWorkingWidth)
    print '8=8'
    DaugmanIrisCenter = DaugmanIrisCenter[::-1]
    #分别计算出眼珠中心的坐标和黑眼仁的半径
    eyecenter = np.float64(DaugmanIrisCenter) * (img.shape[1]/imageWorkingWidth)
    eyeiris =   np.float64(DaugmanIrisRadius) * (img.shape[1]/imageWorkingWidth)
    print '9=9'
    
    ret,imgBinary=cv2.threshold(imggrey,70,255,cv2.THRESH_BINARY)
    print '10=10'
    imgNoHole=fillHoleInBinary(imgfiltered,imgBinary)
    print '11=11'
    #cv2.imshow('imgNoHole_%d_%d'%(count,lor),imgNoHole)
    imgKmeans=UseKmeans(imgNoHole,imggrey)
    if imgKmeans[np.where(imgKmeans==255)].shape[0]<1200:
        return 0,eyecenter,eyeiris,covstackdiffmaxsum,DaugmanIrisCenter,DaugmanIrisRadius,imgfiltered,info
    print '12=12'    
    #计算出上下遮挡比例
    topShelterRate,bottomShelterRate=caculateShelterRate(DaugmanIrisCenter,DaugmanIrisRadius,imgKmeans,count,lor)
    print '13=13'
    
    info.append(DaugmanIrisRadius)
    info.append(topShelterRate)
    info.append(bottomShelterRate)
    
    return 1,eyecenter,eyeiris,covstackdiffmaxsum,DaugmanIrisCenter,DaugmanIrisRadius,imgfiltered,info

def main():
    img = mpimg.imread('/Users/Jiangzp/test.jpg')
    initialize()
    t = time.time()
    for x in range(0,20):
        eyecenter,eyeiris = DaugmanIrisLocalization(img)
    print time.time() -t
    print eyecenter,eyeiris
    
    mypool.close()
    mypool.join()

    plt.imshow(img)
    plt.show()

def localizeIrisAndShow(img):
    initialize()
    eyecenter,eyeiris = DaugmanIrisLocalization(img)
    cv2.circle(img,tuple(eyecenter),int(eyeiris),(0,0,255))
    plt.imshow(img)
    plt.show()


/usr/local/lib/python2.7/site-packages/IPython/nbformat.py:13: ShimWarning: The `IPython.nbformat` package has been deprecated. You should import from nbformat instead.
  "You should import from nbformat instead.", ShimWarning)
/usr/local/lib/python2.7/site-packages/nbformat/current.py:19: UserWarning: nbformat.current is deprecated.

- use nbformat for read/write/validate public API
- use nbformat.vX directly to composing notebooks of a particular version

  """)


importing IPython notebook from GenerateDaugmanTemplateSet.ipynb
importing IPython notebook from DaugmanIrisCore.ipynb
